# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
cities_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['City'],
    alpha = 0.5
)

# Display the map
cities_humidity

In [11]:
### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[(city_data_df['Max Temp'] < 29) & (city_data_df['Max Temp'] > 22) & (city_data_df['Wind Speed'] < 6.5)]

# Drop any rows with null values

my_cities = criteria_df.dropna(how='any')

# Display sample data
# YOUR CODE HERE
my_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
19,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
27,27,bambous virieux,-20.3428,57.7575,22.14,68,75,3.60,MU,1666108246


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = my_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy() 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df ['Hotel Name'] =''
# Display sample data

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
9,saint-pierre,RE,-21.3393,55.4781,68,
19,puerto ayora,EC,-0.7393,-90.3518,82,
20,avarua,CK,-21.2078,-159.7750,73,
27,bambous virieux,MU,-20.3428,57.7575,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1



params = {
    'categories':categories,
    'apiKey':geoapify_key,
    limit:limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = my_cities.loc[index, 'Lat']
    longitude = my_cities.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df    

Starting hotel search
faya - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
bambous virieux - nearest hotel: Otentic Eco Tent Experience
labuhan - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
kapaa - nearest hotel: Pono Kai Resort
mogadishu - nearest hotel: Villa Rays Restaurant & Hotel
hilo - nearest hotel: Dolphin Bay Hotel
businga - nearest hotel: Hôtel Roho
lasa - nearest hotel: Hotel "Drei Kreuz"
vaini - nearest hotel: Keleti International Resort
banda - nearest hotel: #acnindiafy21
caravelas - nearest hotel: Pousada dos Navegantes
cobija - nearest hotel: Hospedaje Novedades
nalut - nearest hotel: No hotel found
mahebourg - nearest hotel: Grand Bel Air
saint-philippe - nearest hotel: Le Baril
ponta do sol - nearest hotel: Estalagem Ponta do Sol
butaritari - nearest hotel: No hotel found
inhambane - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
19,puerto ayora,EC,-0.7393,-90.3518,82,Hostal La Mirada De Solitario George
20,avarua,CK,-21.2078,-159.7750,73,Paradise Inn
27,bambous virieux,MU,-20.3428,57.7575,68,Otentic Eco Tent Experience
...,...,...,...,...,...,...
563,ilhabela,BR,-23.7781,-45.3581,88,Vila Kebaya
566,nhulunbuy,AU,-12.2333,136.7667,78,No hotel found
567,sangamner,IN,19.5667,74.2167,85,Joshi Hotel
569,ranong,TH,9.9658,98.6348,93,Rattana Sin Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['Hotel Name', 'Country'],
    alpha = 0.5
)

# Display the map
hotel_map